### TFM. wikiScraping

In [15]:
import requests
import re
from bs4 import BeautifulSoup
import wget
import urllib.request
import os
import shutil
import json

Creamos la funcion para scrapear Wikiart

In [ ]:
wikiart('francisco-goya')

francisco-goya
https://www.wikiart.org/en/francisco-goya/all-works/text-list



def wikiart(artist):
    url = "https://www.wikiart.org/en/"+artist+"/all-works/text-list"
    page = requests.get(url)
    c=page.content
    soup=BeautifulSoup(c,"html.parser")
    links = []
    for link in soup.findAll('a', attrs={'href': re.compile(r'/en/'+ re.escape(artist)+ r'/')}):
        links.append("https://www.wikiart.org"+link.get('href'))  
    
    images = []
    count = 0
    for link in links:
        try:
            html_page = urllib.request.urlopen(link)
            soup = BeautifulSoup(html_page) 
            for img in soup.findAll('img', {'src':re.compile('.jpg')}):
                images.append(img.get('src'))
        except:
            print("Error al descargar la imagen en el link:", link)
            continue
    
    newpath = r'/home/steiner/Escritorio/Python/pinturas/'
    prefix = artist+"_"
    if not os.path.exists(newpath):
        os.makedirs(newpath)
        
    for image in images:
        try:
            if "banner" not in image and "banner_top" not in image:
                os.chdir(newpath)
                local_image_filename = wget.download(image)  
                shutil.move(local_image_filename,prefix+local_image_filename)
                count += 1
        except:
            print("Error al descargar la imagen:", image)
            continue
    print("Se han descargado", count, "imágenes.")


In [16]:
def wikiart(artist):
    url = "https://www.wikiart.org/en/"+artist+"/all-works/text-list"
    print(artist)
    print(url)
    page = requests.get(url)
    c=page.content
    soup=BeautifulSoup(c,"html.parser")
    links = []
    for link in soup.findAll('a', attrs={'href': re.compile(r'/en/'+ re.escape(artist)+ r'/')}):
        links.append("https://www.wikiart.org"+link.get('href'))  
    images = []
    metadata = []
    for link in links:
        html_page = urllib.request.urlopen(link)
        soup = BeautifulSoup(html_page) 
        for img in soup.findAll('img', {'src':re.compile('.jpg')}):
            image = img.get('src')
            if "banner" not in image and "banner_top" not in image:
                images.append(image)
                style = soup.find("a", text="estilo")
                if style:
                    style = style.text
                else:
                    style = None

                genre = soup.find("span", itemprop="genre")
                if genre:
                    genre = genre.text
                else:
                    genre = None

                media = soup.find("a", text="fresco")
                if media:
                    media = media.text
                else:
                    media = None

                date = soup.find("span", itemprop="dateCreated")
                if date:
                    date = date.text
                else:
                    date = None

                metadata.append({
                    "style": style,
                    "genre": genre,
                    "media": media,
                    "url": link,
                    "date": date
                })

    newpath = r'/home/steiner/Escritorio/Python/pinturas/'
    prefix = artist+"_"
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    counter = 0
    for image in images:
        os.chdir(newpath)
        local_image_filename = wget.download(image)  
        shutil.move(local_image_filename,str(counter)+"_"+local_image_filename)
        counter += 1

    with open(newpath+"/metadata.json", "w") as f:
        json.dump(metadata, f)

